Оценки на Kaggle (меньше-лучше)

https://www.kaggle.com/competitions/california-house-prices/overview

![](kaggle-scores.png)

In [1]:
import pandas as pd
import xgboost as xg
df: pd.DataFrame = pd.read_csv("train.csv")
df.describe()

,Id,Sold Price,Year built,Lot,Bathrooms,Full bathrooms,Total interior livable area,Total spaces,Garage spaces,Elementary School Score,Elementary School Distance,Middle School Score,Middle School Distance,High School Score,High School Distance,Tax assessed value,Annual tax amount,Listed Price,Last Sold Price,Zip
count,47439.000000,4.743900e+04,46394.000000,3.325800e+04,43974.000000,39574.000000,4.491300e+04,46523.000000,46522.000000,42543.000000,42697.000000,30734.000000,30735.000000,42220.000000,42438.000000,4.378700e+04,43129.000000,4.743900e+04,2.967300e+04,47439.000000
mean,23719.000000,1.296050e+06,1956.634888,2.353383e+05,2.355642,2.094961,5.774587e+03,1.567117,1.491746,5.720824,1.152411,5.317206,1.691593,6.134344,2.410366,7.863118e+05,9956.843817,1.315890e+06,8.078537e+05,93279.178587
std,13694.604047,1.694452e+06,145.802456,1.192507e+07,1.188805,0.963320,8.324363e+05,9.011608,8.964319,2.103350,2.332367,2.002768,2.462879,1.984711,3.596120,1.157796e+06,13884.254976,2.628695e+06,1.177903e+06,2263.459104
min,0.000000,1.005000e+05,0.000000,0.000000e+00,0.000000,1.000000,1.000000e+00,-15.000000,-15.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,85611.000000
25%,11859.500000,5.650000e+05,1946.000000,4.991000e+03,2.000000,2.000000,1.187000e+03,0.000000,0.000000,4.000000,0.300000,4.000000,0.600000,5.000000,0.800000,2.549615e+05,3467.000000,5.745000e+05,3.350000e+05,90220.000000
50%,23719.000000,9.600000e+05,1967.000000,6.502000e+03,2.000000,2.000000,1.566000e+03,1.000000,1.000000,6.000000,0.500000,5.000000,1.000000,6.000000,1.300000,5.475240e+05,7129.000000,9.490000e+05,5.980000e+05,94114.000000
75%,35578.500000,1.525000e+06,1989.000000,1.045400e+04,3.000000,2.000000,2.142000e+03,2.000000,2.000000,7.000000,1.000000,7.000000,1.800000,8.000000,2.400000,9.371625e+05,12010.000000,1.498844e+06,9.500000e+05,95073.000000
max,47438.000000,9.000000e+07,9999.000000,1.897474e+09,24.000000,17.000000,1.764164e+08,1000.000000,1000.000000,10.000000,57.200000,9.000000,57.200000,10.000000,73.900000,4.590000e+07,552485.000000,4.025320e+08,9.000000e+07,96155.000000


In [2]:
df.head(3)

,Id,Address,Sold Price,Summary,Type,Year built,Heating,Cooling,Parking,Lot,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,0,540 Pine Ln,3825000.0,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,...,"Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,NaN,NaN,Los Altos,94022,CA
1,1,1727 W 67th St,505000.0,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,...,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047,CA
2,2,28093 Pine Ave,140000.0,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958.0,Forced air,NaN,0 spaces,9147.0,...,NaN,49627.0,468.0,2019-08-25,180000.0,NaN,NaN,Strawberry,95375,CA


In [3]:
x_train = df[["Year built", "Tax assessed value", "Annual tax amount", "Listed Price"]]
y_train = df["Sold Price"]

In [4]:
x_train.isna().sum()

Year built            1045
Tax assessed value    3652
Annual tax amount     4310
Listed Price             0
dtype: int64

In [5]:
x_train = x_train.fillna(x_train.mean())

In [6]:
df_test = pd.read_csv("test.csv")
df_test.head(3)

,Id,Address,Summary,Type,Year built,Heating,Cooling,Parking,Lot,Bedrooms,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,47439,3034 N Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,NaN,NaN,2020-11-06,799900.0,2020-07-01,819000.0,Dodgertown,90090,CA
1,47440,565 Kenilworth Ave,duplex fixer. Input for comps only,SingleFamily,1924.0,Natural Gas,NaN,Detached,10018.8,3,...,Detached,521977.0,7494.0,2014-04-04,479950.0,2020-11-03,15000.0,San Leandro,94577,CA
2,47441,3028 N Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,NaN,NaN,2020-11-12,839900.0,NaN,NaN,Los Angeles,90039,CA


In [7]:
x_test = df_test[["Year built", "Tax assessed value", "Annual tax amount", "Listed Price"]]
x_test = x_test.fillna(x_train.mean())

In [8]:
model = xg.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [9]:
y_predict = model.predict(x_test)

In [10]:
def make_valid_submission(y_predict, submission_name="submission"):
    df_predict = pd.DataFrame.from_dict(y_predict)
    df_predict.insert(0, 'Id', df_test["Id"])
    df_predict = df_predict.set_axis(['Id', 'Sold Price'], axis=1)    
    df_predict.to_csv(f"{submission_name}.csv", index=False, encoding='utf-8')        

In [11]:
make_valid_submission(y_predict, "XGBRegressor")

In [12]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute
from sklearn.metrics import r2_score
from numpy import arange
from sklearn.model_selection import GridSearchCV

In [13]:
from sklearn.linear_model import LogisticRegression, LinearRegression
import numpy as np
from numpy import mean
from numpy import std
from sklearn.metrics import classification_report, f1_score, fbeta_score, roc_auc_score, confusion_matrix, accuracy_score

In [14]:
def model_cross_val_score(model, dataset_pairs, scorings, silent=False):    
    for pair in dataset_pairs:    
        if not silent:
            print(f"\n==> Result on {pair[2]} dataset:")
        for scoring in scorings:            
            cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
            scores = cross_val_score(model, pair[0], pair[1], scoring=scoring, cv=cv, n_jobs=-1)
            scores = absolute(scores)
            if not silent:                    
                print(f"{scoring}: mean {mean(scores):1.4f} std {std(scores):1.4f}")                          

In [15]:
def grid_search(model_class, X, y, scoring, grid, silent=False):
    first_values_dict = {k: v[0] for k, v in grid.items()}
    model = model_class(**first_values_dict)    
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)        
    search = GridSearchCV(model, grid, scoring=scoring, cv=cv, n_jobs=-1)
    results = search.fit(X, y)
    if not silent:
        print(f"Best {scoring} score: {results.best_score_:1.4f}")
        print(f"Config: {results.best_params_}")
    return results.best_params_

In [16]:
def grid_search_w_cross_val(model, dataset_pairs, validate_scorings, grid_scoring, grid, val_on_same_dataset = False):
    print(f"Searching parameters for {model.__name__} model.")
    
    for pair in dataset_pairs:            
        
        print(f"{'=' * 50}\nGrid Search result on {pair[2]} dataset: ")
        params = grid_search(model, pair[0], pair[1], scoring=grid_scoring, grid=grid)
        
        print(f"\nValidation result: ")        
        val_pairs = [pair] if val_on_same_dataset else dataset_pairs            
        model_cross_val_score(model(**params), val_pairs, scorings=validate_scorings)

In [17]:
len(y_train[y_train <=0])

0

In [18]:
lr_grid = dict()
lr_grid['fit_intercept'] = [True, False]
lr_scorings = ['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error']  # neg_mean_squared_log_error # neg_mean_squared_error
lr_scoring = 'neg_mean_squared_error'
lr = LinearRegression(**grid_search(LinearRegression, x_train, y_train, lr_scoring, lr_grid, silent=True))

In [19]:
grid_search_w_cross_val(LinearRegression, [(x_train, y_train, "Base")], lr_scorings, lr_scoring, lr_grid)

Searching parameters for LinearRegression model.
Grid Search result on Base dataset: 
Best neg_mean_squared_error score: -2546151748538.7168
Config: {'fit_intercept': True}

Validation result: 

==> Result on Base dataset:
r2: mean 0.8521 std 0.5127
neg_mean_absolute_error: mean 392673.5073 std 71229.9658
neg_mean_squared_error: mean 2546151748538.7168 std 2908784730655.6143


In [20]:
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
make_valid_submission(lr_pred, "LinearRegression")

In [21]:
from sklearn.ensemble import RandomForestRegressor

rfr_grid=dict()
rfr_grid['n_estimators'] = arange(5, 15, 2)
rf = RandomForestRegressor(**grid_search(RandomForestRegressor, x_train, y_train, "neg_mean_squared_error", rfr_grid, silent=True))

In [22]:
rf.fit(x_train, y_train)
rf_predict = rf.predict(x_test)
make_valid_submission(rf_predict, "RandomForestRegressor")

In [23]:
grid_search_w_cross_val(RandomForestRegressor, [(x_train, y_train, "Base")], lr_scorings, lr_scoring, rfr_grid)

Searching parameters for RandomForestRegressor model.
Grid Search result on Base dataset: 
Best neg_mean_squared_error score: -717803550829.3270
Config: {'n_estimators': 11}

Validation result: 

==> Result on Base dataset:
r2: mean 0.7479 std 0.0954
neg_mean_absolute_error: mean 161900.9527 std 4847.8841
neg_mean_squared_error: mean 704003610258.4183 std 319693073334.4158


Случайный лес
комитет
линейная регрессия

In [24]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from IPython.display import clear_output

In [25]:
from sklearn.ensemble import VotingRegressor

reg_grid = dict()
reg_grid['alpha'] =  np.arange(0.2, 1, 0.1)
reg_scorings = ['neg_mean_absolute_error', 'r2']
reg_scoring = 'r2'

knn_grid = dict()
knn_grid['n_neighbors'] = arange(1, 20, 5)
knn_grid['metric'] = ['cityblock', 'euclidean', 'manhattan', 'minkowski']
knn_scorings = ['f1', 'accuracy', 'r2']
knn_scoring = 'f1'

VotingRegressor
vr_grid=dict()
vr_grid['estimators'] = [ ('KNeighborsRegressor', KNeighborsRegressor(**grid_search(KNeighborsRegressor, x_train, y_train, "r2", knn_grid, silent=True)) ),
                          ('RandomForestRegressor', RandomForestRegressor(**grid_search(RandomForestRegressor, x_train, y_train, "r2", rfr_grid, silent=True)) ),
                        ]

clear_output()

In [26]:
vr = VotingRegressor(estimators=vr_grid['estimators'])
vr.fit_transform(x_train.to_numpy(), y_train)
vr_predict = vr.predict(x_test)
make_valid_submission(vr_predict, "VotingRegressor")
clear_output() 

In [27]:
from sklearn.linear_model import LogisticRegression

lr_grid = dict()
lr_grid['max_iter'] = np.arange(50, 150, 50)
lr_grid['solver'] = ['sag', 'saga']
lr_scorings = ['f1', 'accuracy', 'r2']
lr_scoring = 'f1'

In [28]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [29]:
make_valid_submission(lr.predict(x_test), "LinearRegression")

In [30]:
lgr = LogisticRegression(solver='saga', max_iter=10)
lgr.fit(x_train, y_train)

/home/raiden/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=10, solver='saga')

In [31]:
make_valid_submission(lgr.predict(x_test), "LogisticRegression")